# Runtime context

In [8]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [9]:
from dataclasses import dataclass


# Setup your agent runtime context
@dataclass
class UserInfo:
    first: str
    last: str

In [10]:
import re
from dataclasses import asdict

from langchain_core.tools import tool
from langgraph.runtime import get_runtime


@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results.

    Named parameters like :first and :last will be filled from runtime context.
    """
    runtime = get_runtime(UserInfo)
    try:
        placeholders = re.findall(r":(\w+)", query)

        if placeholders:
            return db.run(query, parameters=asdict(runtime.context or {}))
        else:
            return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [11]:
import requests
from langchain_core.tools import tool

FALLBACK_RATES = {"USD": 1.0, "EUR": 0.92, "JPY": 150.0, "GBP": 0.79}


@tool
def convert_currency(amount: float, to_currency: str) -> float:
    """Convert an amount in USD to another currency using live exchange rates.

    Always use this tool if the user requests a currency different
    from the one stored in the database (USD).
    """
    try:
        url = f"https://api.exchangerate.host/convert?from=USD&to={to_currency}&amount={amount}"
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        data = response.json()
        result = data.get("result")
        if result is not None:
            return round(float(result), 2)
    except Exception as e:
        print(f"[convert_currency] Falling back due to error: {e}")

    if to_currency not in FALLBACK_RATES:
        raise ValueError("Unsupported currency in fallback mode")
    return round(amount * FALLBACK_RATES[to_currency], 2)

In [12]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
- If a query involves a specific customer, always use the named parameters
  :first and :last (instead of hard-coding values).
  Example: SELECT InvoiceId, Total FROM Invoice
           JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
           WHERE Customer.FirstName = :first AND Customer.LastName = :last;
  Assume the customer first and last name are known and will be inserted into the query later.
"""

In [14]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5",
    tools=[execute_sql, convert_currency],
    system_prompt=SYSTEM_PROMPT,
    context_schema=UserInfo,
)

In [16]:
question = "What is the most costly purchase I made?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=UserInfo(first="Frank", last="Harris"),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase I made?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_RFCohuWssCJrXspzFMAWflAi)
 Call ID: call_RFCohuWssCJrXspzFMAWflAi
  Args:
    query: SELECT i.InvoiceId, i.InvoiceDate, i.Total, COUNT(il.InvoiceLineId) AS LineItemCount
FROM Invoice AS i
JOIN Customer AS c ON i.CustomerId = c.CustomerId
JOIN InvoiceLine AS il ON il.InvoiceId = i.InvoiceId
WHERE c.FirstName = :first AND c.LastName = :last
GROUP BY i.InvoiceId, i.InvoiceDate, i.Total
ORDER BY i.Total DESC
LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

[(145, '2010-09-23 00:00:00', 13.86, 14)]
================================== Ai Message ==================================

Your most costly purchase was $13.86 on 2010-09-23 (Invoice #145), with 14 items.
